In [1]:
import pandas as pd

In [2]:
bookings_df = pd.read_csv("./dataset/bookings.csv")
customers_df = pd.read_csv("./dataset/customers.csv")
drivers_df = pd.read_csv("./dataset/drivers.csv")
location_demand_df = pd.read_csv("./dataset/location_demand.csv")
time_features_df = pd.read_csv("./dataset/time_features.csv")

<h1><b>Understanding data before touching</b></h1>

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
bookings_df

,booking_id,booking_date,booking_time,day_of_week,is_weekend,hour_of_day,city,pickup_location,drop_location,vehicle_type,ride_distance_km,estimated_ride_time_min,actual_ride_time_min,traffic_level,weather_condition,base_fare,surge_multiplier,booking_value,booking_status,incomplete_ride_reason,customer_id,driver_id
0,B_000001,2025-12-11,00:07:00,Thursday,0,0,Mumbai,Loc_19,Loc_16,Bike,7.01,46.30,NaN,High,Heavy Rain,76.12,2.0,148.22,Cancelled,NaN,C_005097,D_004592
1,B_000002,2025-07-07,06:13:00,Monday,0,6,Mumbai,Loc_32,Loc_38,Cab,9.67,43.54,42.28,Medium,Heavy Rain,254.15,1.8,465.85,Completed,NaN,C_008459,D_000148
2,B_000003,2025-08-23,08:53:00,Saturday,1,8,Chennai,Loc_28,Loc_1,Auto,16.18,48.55,NaN,Low,Heavy Rain,234.20,1.9,457.03,Cancelled,NaN,C_003471,D_004976
3,B_000004,2025-04-12,10:25:00,Saturday,1,10,Delhi,Loc_16,Loc_30,Bike,1.02,4.61,4.76,Medium,Rain,28.20,1.8,51.03,Completed,NaN,C_002161,D_001173
4,B_000005,2025-08-23,00:08:00,Saturday,1,0,Hyderabad,Loc_22,Loc_31,Bike,12.35,55.56,64.53,Medium,Clear,118.77,1.2,144.73,Completed,NaN,C_005617,D_001175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,B_099996,2025-02-09,07:32:00,Sunday,1,7,Chennai,Loc_27,Loc_10,Auto,12.32,55.42,59.31,Medium,Rain,187.79,1.5,292.05,Completed,NaN,C_006714,D_004332
99996,B_099997,2025-06-02,08:45:00,Monday,0,8,Mumbai,Loc_43,Loc_2,Auto,9.58,63.25,75.22,High,Rain,155.00,2.0,300.66,Completed,NaN,C_002615,D_002143
99997,B_099998,2025-08-20,03:36:00,Wednesday,0,3,Bangalore,Loc_20,Loc_38,Auto,7.57,34.06,31.32,Medium,Heavy Rain,130.84,1.8,241.33,Completed,NaN,C_002709,D_002794
99998,B_099999,2025-07-30,23:04:00,Wednesday,0,23,Hyderabad,Loc_5,Loc_3,Cab,22.87,150.95,174.40,High,Clear,491.68,1.4,662.67,Completed,NaN,C_007985,D_002492


In [5]:
customers_df

,customer_id,customer_gender,customer_age,customer_city,customer_signup_days_ago,preferred_vehicle_type,total_bookings,completed_rides,cancelled_rides,incomplete_rides,cancellation_rate,avg_customer_rating,customer_cancel_flag
0,C_000001,Non-Binary,56,Bangalore,556,Cab,10,9,1,0,0.100000,3.8,0
1,C_000002,Male,46,Bangalore,82,Bike,8,7,1,0,0.125000,4.1,0
2,C_000003,Female,32,Delhi,969,Bike,7,4,1,2,0.142857,3.8,0
3,C_000004,Female,60,Hyderabad,549,Bike,11,7,3,1,0.272727,3.7,1
4,C_000005,Non-Binary,25,Hyderabad,359,Bike,9,5,3,1,0.333333,3.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,C_009996,Non-Binary,33,Bangalore,594,Cab,6,3,2,1,0.333333,4.1,1
9996,C_009997,Female,29,Delhi,843,Bike,12,10,1,1,0.083333,4.8,0
9997,C_009998,Male,18,Bangalore,74,Auto,8,7,1,0,0.125000,4.3,0
9998,C_009999,Non-Binary,25,Delhi,806,Cab,9,6,2,1,0.222222,4.3,1


In [6]:
drivers_df

,driver_id,driver_age,driver_city,vehicle_type,driver_experience_years,total_assigned_rides,accepted_rides,incomplete_rides,delay_count,acceptance_rate,delay_rate,avg_driver_rating,avg_pickup_delay_min,driver_delay_flag
0,D_000001,39,Bangalore,Auto,1,25,16,0,0,0.64,0.00,4.1,2.0,0
1,D_000002,40,Chennai,Cab,2,14,11,3,2,0.79,0.14,4.8,6.1,1
2,D_000003,26,Bangalore,Auto,12,19,14,3,2,0.74,0.11,4.1,2.6,1
3,D_000004,46,Chennai,Auto,3,18,13,2,2,0.72,0.11,4.2,2.6,1
4,D_000005,32,Chennai,Bike,7,18,16,1,1,0.89,0.06,4.8,4.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,D_004996,28,Delhi,Auto,8,16,9,1,0,0.56,0.00,4.8,3.0,0
4996,D_004997,42,Hyderabad,Cab,3,18,12,0,0,0.67,0.00,4.7,3.0,0
4997,D_004998,33,Bangalore,Bike,7,22,19,2,0,0.86,0.00,4.4,2.0,0
4998,D_004999,36,Bangalore,Bike,3,26,20,1,1,0.77,0.04,4.8,2.6,0


In [7]:
location_demand_df

,city,pickup_location,hour_of_day,vehicle_type,total_requests,completed_rides,cancelled_rides,avg_wait_time_min,avg_surge_multiplier,demand_level
0,Bangalore,Loc_1,0,Auto,2,2,0,87.940000,1.400000,Low
1,Bangalore,Loc_1,0,Bike,5,5,0,68.088000,1.460000,Low
2,Bangalore,Loc_1,0,Cab,6,5,0,50.913333,1.733333,Medium
3,Bangalore,Loc_1,1,Auto,3,2,0,72.883333,1.566667,Low
4,Bangalore,Loc_1,1,Bike,7,6,1,33.374286,1.242857,Medium
...,...,...,...,...,...,...,...,...,...,...
17936,Mumbai,Loc_9,22,Bike,12,9,3,49.346667,1.500000,Medium
17937,Mumbai,Loc_9,22,Cab,2,2,0,93.575000,1.200000,Low
17938,Mumbai,Loc_9,23,Auto,4,3,1,46.400000,1.475000,Low
17939,Mumbai,Loc_9,23,Bike,4,2,2,46.977500,1.500000,Low


In [8]:
time_features_df

,datetime,hour_of_day,day_of_week,is_weekend,is_holiday,peak_time_flag,season
0,2025-01-01 00:00:00,0,Wednesday,0,0,0,Winter
1,2025-01-01 01:00:00,1,Wednesday,0,0,0,Winter
2,2025-01-01 02:00:00,2,Wednesday,0,0,0,Winter
3,2025-01-01 03:00:00,3,Wednesday,0,0,0,Winter
4,2025-01-01 04:00:00,4,Wednesday,0,0,0,Winter
...,...,...,...,...,...,...,...
8755,2025-12-31 19:00:00,19,Wednesday,0,0,1,Winter
8756,2025-12-31 20:00:00,20,Wednesday,0,0,1,Winter
8757,2025-12-31 21:00:00,21,Wednesday,0,0,0,Winter
8758,2025-12-31 22:00:00,22,Wednesday,0,0,0,Winter


In [9]:
# bookings_df.isna().sum(), bookings_df.isna().sum()

print("\n\n------bookings_df-----")
print(f"Duplicates: {bookings_df.duplicated().sum()}")
print(bookings_df.isna().sum())

print("\n\n------customers_df-----")
print(f"Duplicates: {customers_df.duplicated().sum()}")
print(customers_df.isna().sum())

print("\n\n------drivers_df-----")
print(f"Duplicates: {drivers_df.duplicated().sum()}")
print(drivers_df.isna().sum())

print("\n\n------location_demand_df-----")
print(f"Duplicates: {location_demand_df.duplicated().sum()}")
print(location_demand_df.isna().sum())

print("\n\n------time_features_df-----")
print(f"Duplicates: {time_features_df.duplicated().sum()}")
print(time_features_df.isna().sum())



------bookings_df-----
Duplicates: 0
booking_id                     0
booking_date                   0
booking_time                   0
day_of_week                    0
is_weekend                     0
hour_of_day                    0
city                           0
pickup_location                0
drop_location                  0
vehicle_type                   0
ride_distance_km               0
estimated_ride_time_min        0
actual_ride_time_min       31654
traffic_level                  0
weather_condition              0
base_fare                      0
surge_multiplier               0
booking_value                  0
booking_status                 0
incomplete_ride_reason     91630
customer_id                    0
driver_id                      0
dtype: int64


------customers_df-----
Duplicates: 0
customer_id                 0
customer_gender             0
customer_age                0
customer_city               0
customer_signup_days_ago    0
preferred_vehicle_type      0
to

In [10]:
bookings_df.columns, customers_df.columns, drivers_df.columns, location_demand_df.columns, time_features_df.columns

(Index(['booking_id', 'booking_date', 'booking_time', 'day_of_week',
        'is_weekend', 'hour_of_day', 'city', 'pickup_location', 'drop_location',
        'vehicle_type', 'ride_distance_km', 'estimated_ride_time_min',
        'actual_ride_time_min', 'traffic_level', 'weather_condition',
        'base_fare', 'surge_multiplier', 'booking_value', 'booking_status',
        'incomplete_ride_reason', 'customer_id', 'driver_id'],
       dtype='object'),
 Index(['customer_id', 'customer_gender', 'customer_age', 'customer_city',
        'customer_signup_days_ago', 'preferred_vehicle_type', 'total_bookings',
        'completed_rides', 'cancelled_rides', 'incomplete_rides',
        'cancellation_rate', 'avg_customer_rating', 'customer_cancel_flag'],
       dtype='object'),
 Index(['driver_id', 'driver_age', 'driver_city', 'vehicle_type',
        'driver_experience_years', 'total_assigned_rides', 'accepted_rides',
        'incomplete_rides', 'delay_count', 'acceptance_rate', 'delay_rate',
     

<h1><b>EDA</b></h1>

In [11]:
import plotly.express as px

fig = px.histogram(bookings_df["base_fare"], nbins=50)
fig.show()

fig = px.histogram(bookings_df["surge_multiplier"])
fig.show()

fig = px.histogram(bookings_df["booking_value"], nbins=50)
fig.show()

In [12]:
fig = px.sunburst(
    bookings_df,
    path=['booking_status', 'weather_condition', 'traffic_level'], 
    title="Booking Cancellations across weather condition and traffic level"
)
fig.show()

fig = px.sunburst(
    bookings_df,
    path=['booking_status', 'city'], 
    title="Booking Cancellations"
)
fig.show()

<h1><b>Handling null values in bookings_df</b></h1>

In [13]:
mask_not_completed = (bookings_df["booking_status"] != "Completed") & (bookings_df["incomplete_ride_reason"].isna())
mask_completed = (bookings_df["booking_status"] == "Completed") & (bookings_df["incomplete_ride_reason"].isna())

bookings_df.loc[mask_not_completed, "incomplete_ride_reason"] = "Unknown"
bookings_df.loc[mask_completed, "incomplete_ride_reason"] = "Not Applicable"

mask_actual_ride_time_min = (bookings_df["booking_status"] != "Completed") & bookings_df["actual_ride_time_min"].isna()

bookings_df.loc[mask_actual_ride_time_min, "actual_ride_time_min"] = 0

bookings_df.isna().sum()

booking_id                 0
booking_date               0
booking_time               0
day_of_week                0
is_weekend                 0
hour_of_day                0
city                       0
pickup_location            0
drop_location              0
vehicle_type               0
ride_distance_km           0
estimated_ride_time_min    0
actual_ride_time_min       0
traffic_level              0
weather_condition          0
base_fare                  0
surge_multiplier           0
booking_value              0
booking_status             0
incomplete_ride_reason     0
customer_id                0
driver_id                  0
dtype: int64

<h1><b>Feature Engineering<b></h1>

In [14]:
bookings_df["booking_date"] = pd.to_datetime(bookings_df["booking_date"])
bookings_df["booking_time"] = pd.to_datetime(bookings_df["booking_time"], format="%H:%M:%S")

bookings_df["day"] = bookings_df["booking_date"].dt.day
bookings_df["month"] = bookings_df["booking_date"].dt.month
bookings_df["minutes"] = bookings_df["booking_time"].dt.minute

In [15]:
from utils import identify_rush_hour

bookings_df["rush_hour_flag"] = bookings_df.apply(identify_rush_hour,axis=1)
bookings_df["rush_hour_flag"].value_counts()

bookings_df["fare_per_km"] = bookings_df["booking_value"] / bookings_df["ride_distance_km"]

bookings_df["fare_per_min"] = bookings_df["booking_value"] / bookings_df["estimated_ride_time_min"]

bookings_df["long_distance_flag"] = (bookings_df["ride_distance_km"] > 15).astype(int)

<h1><b>Combining Dataset</b></h1>

In [16]:
# Renaming columns to avoid confusion and redundency
customers_df = customers_df.rename(columns={"incomplete_rides": "customer_incomplete_rides", "completed_rides" : "customer_completed_rides", "cancelled_rides" : "customer_cancelled_rides", "total_bookings": "customer_total_bookings", "cancellation_rate": "customer_cancellation_rate"})

drivers_df = drivers_df.rename(columns={"vehicle_type": "driver_vehicle_type", "incomplete_rides": "driver_incomplete_rides"})

# completed_rides', 'cancelled_rides
location_demand_df = location_demand_df.rename(columns={"completed_rides": "location_completed_rides", "cancelled_rides": "location_cancelled_rides"})

In [17]:
final_df = (
    bookings_df
        .merge(customers_df, on="customer_id", how="left")
        .merge(drivers_df, on="driver_id", how="left")
        .merge(location_demand_df, on=["city", "pickup_location", "hour_of_day", "vehicle_type"],how="left") 
)
final_df

,booking_id,booking_date,booking_time,day_of_week,is_weekend,hour_of_day,city,pickup_location,drop_location,vehicle_type,ride_distance_km,estimated_ride_time_min,actual_ride_time_min,traffic_level,weather_condition,base_fare,surge_multiplier,booking_value,booking_status,incomplete_ride_reason,customer_id,driver_id,day,month,minutes,rush_hour_flag,fare_per_km,fare_per_min,long_distance_flag,customer_gender,customer_age,customer_city,customer_signup_days_ago,preferred_vehicle_type,customer_total_bookings,customer_completed_rides,customer_cancelled_rides,customer_incomplete_rides,customer_cancellation_rate,avg_customer_rating,customer_cancel_flag,driver_age,driver_city,driver_vehicle_type,driver_experience_years,total_assigned_rides,accepted_rides,driver_incomplete_rides,delay_count,acceptance_rate,delay_rate,avg_driver_rating,avg_pickup_delay_min,driver_delay_flag,total_requests,location_completed_rides,location_cancelled_rides,avg_wait_time_min,avg_surge_multiplier,demand_level
0,B_000001,2025-12-11,1900-01-01 00:07:00,Thursday,0,0,Mumbai,Loc_19,Loc_16,Bike,7.01,46.30,0.00,High,Heavy Rain,76.12,2.0,148.22,Cancelled,Unknown,C_005097,D_004592,11,12,7,0,21.144080,3.201296,0,Male,60,Chennai,548,Bike,16,13,3,0,0.187500,4.3,0,52,Chennai,Cab,10,18,16,2,1,0.89,0.06,4.9,2.9,0,4,3,1,51.002500,1.525000,Low
1,B_000002,2025-07-07,1900-01-01 06:13:00,Monday,0,6,Mumbai,Loc_32,Loc_38,Cab,9.67,43.54,42.28,Medium,Heavy Rain,254.15,1.8,465.85,Completed,Not Applicable,C_008459,D_000148,7,7,13,0,48.174767,10.699357,0,Female,26,Chennai,848,Cab,10,10,0,0,0.000000,4.7,0,32,Mumbai,Auto,5,18,14,2,1,0.78,0.06,4.6,3.9,0,4,4,0,47.202500,1.575000,Low
2,B_000003,2025-08-23,1900-01-01 08:53:00,Saturday,1,8,Chennai,Loc_28,Loc_1,Auto,16.18,48.55,0.00,Low,Heavy Rain,234.20,1.9,457.03,Cancelled,Unknown,C_003471,D_004976,23,8,53,0,28.246601,9.413594,1,Female,55,Bangalore,161,Cab,6,2,3,1,0.500000,3.7,1,38,Mumbai,Cab,14,27,22,4,4,0.81,0.15,4.4,5.2,1,7,4,3,68.342857,1.685714,Medium
3,B_000004,2025-04-12,1900-01-01 10:25:00,Saturday,1,10,Delhi,Loc_16,Loc_30,Bike,1.02,4.61,4.76,Medium,Rain,28.20,1.8,51.03,Completed,Not Applicable,C_002161,D_001173,12,4,25,0,50.029412,11.069414,0,Female,36,Hyderabad,322,Cab,11,8,2,1,0.181818,3.7,0,45,Delhi,Cab,14,25,17,3,2,0.68,0.08,4.2,3.2,0,3,2,1,27.963333,1.566667,Low
4,B_000005,2025-08-23,1900-01-01 00:08:00,Saturday,1,0,Hyderabad,Loc_22,Loc_31,Bike,12.35,55.56,64.53,Medium,Clear,118.77,1.2,144.73,Completed,Not Applicable,C_005617,D_001175,23,8,8,0,11.719028,2.604932,0,Male,30,Delhi,855,Cab,13,11,0,2,0.000000,4.4,0,54,Delhi,Auto,4,15,12,1,1,0.80,0.07,4.5,4.0,0,8,7,1,91.583750,1.400000,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,B_099996,2025-02-09,1900-01-01 07:32:00,Sunday,1,7,Chennai,Loc_27,Loc_10,Auto,12.32,55.42,59.31,Medium,Rain,187.79,1.5,292.05,Completed,Not Applicable,C_006714,D_004332,9,2,32,0,23.705357,5.269758,0,Female,64,Delhi,507,Cab,13,7,5,1,0.384615,4.1,1,34,Hyderabad,Cab,6,17,15,0,0,0.88,0.00,4.1,2.0,0,8,4,4,78.031250,1.687500,Medium
99996,B_099997,2025-06-02,1900-01-01 08:45:00,Monday,0,8,Mumbai,Loc_43,Loc_2,Auto,9.58,63.25,75.22,High,Rain,155.00,2.0,300.66,Completed,Not Applicable,C_002615,D_002143,2,6,45,1,31.384134,4.753518,0,Female,25,Bangalore,348,Cab,12,8,4,0,0.333333,3.6,1,38,Delhi,Cab,9,23,19,1,1,0.83,0.04,4.6,1.6,0,7,7,0,84.765714,1.857143,Medium
99997,B_099998,2025-08-20,1900-01-01 03:36:00,Wednesday,0,3,Bangalore,Loc_20,Loc_38,Auto,7.57,34.06,31.32,Medium,Heavy Rain,130.84,1.8,241.33,Completed,Not Applicable,C_002709,D_002794,20,8,36,0,31.879789,7.085437,0,Male,38,Delhi,845,Bike,12,7,3,2,0.250000,4.7,1,33,Hyderabad,Auto,7,24,20,5,3,0.83,0.12,4.4,2.8,1,6,5,1,57.021667,1.733333,Medium
99998,B_099999,2025-07-30,1900-01-01 23:04:00,Wednesday,0,23,Hyderabad,Loc_5,Loc_3,Cab,22.87,150.95,174.40,High,Cle

In [ ]:
final_df.to_csv("./dataset/final_dataset.csv", index=False)